This notebook inspired from: http://www.austintaylor.io/d3/python/pandas/2016/02/01/create-d3-chart-python-force-directed/

In [ ]:
import pandas as pd
import json
import re
import numpy as np

def opposite(value, choices):
    return set(choices).difference(set([value])).pop()

### Generate some data to work with

Time = np.random.rand(100) * np.random.rand(100) * 2
Source = np.random.choice(['10.25.22.253','10.25.22.250'],100)
Src_Port = np.random.choice([2546,80],100)
df = pd.DataFrame({'Time':Time,'Source':Source,'Src_Port':Src_Port})
df['Destination'] = df['Source'].apply(lambda x: opposite(x, ['10.25.22.253','10.25.22.250']))
df['Dst_Port'] = df['Src_Port'].apply(lambda x: opposite(x, [2546,80]))
df['Protocol'] = np.random.choice(['TCP','HTTP'],100)
df['Length'] = np.random.randint(60,14654,100)
df

src_dst = df[["Source","Destination"]]

src_dst.sample(10)

def ip_matcher(address):
    # Used to validate if string is an ipaddress
    ip = re.match(
        '^(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)$', address)
    if ip:
        return True
    else:
        return False

src_dst.rename(columns={"Source":"source","Destination":"target"}, inplace=True)
src_dst['valid_src'] = src_dst.source.apply(ip_matcher)
src_dst['valid_target'] = src_dst.target.apply(ip_matcher)

valid_src_dest = src_dst[(src_dst.valid_src==True) & (src_dst.valid_target==True)]

grouped_src_dst = valid_src_dest.groupby(["source","target"]).size().reset_index()

unique_ips = pd.Index(grouped_src_dst['source']
                      .append(grouped_src_dst['target'])
                      .reset_index(drop=True).unique())

group_dict = {}
counter = 0
for ip in unique_ips:
    breakout_ip = re.match("^(\d{1,3})\.(\d{1,3})\.(\d{1,3})\.(\d{1,3})$", ip)
    if breakout_ip:
        net_id = '.'.join(breakout_ip.group(1,2,3))
        if net_id not in group_dict:
            counter += 1
            group_dict[net_id] = counter
        else:
            pass

grouped_src_dst.rename(columns={0:'count'}, inplace=True)
temp_links_list = list(grouped_src_dst.apply(lambda row: {"source": row['source'], "target": row['target'], "value": row['count']}, axis=1))

temp_links_list

links_list = []
for link in temp_links_list:
    record = {"value":link['value'], "source":unique_ips.get_loc(link['source']),
     "target": unique_ips.get_loc(link['target'])}
    links_list.append(record)

nodes_list = []

for ip in unique_ips:
    breakout_ip = re.match("^(\d{1,3})\.(\d{1,3})\.(\d{1,3})\.(\d{1,3})$", ip)
    if breakout_ip:
        net_id = '.'.join(breakout_ip.group(1,2,3))
        nodes_list.append({"name":ip, "group": group_dict.get(net_id)})

nodes_list

links_list